<a href="https://colab.research.google.com/github/Lipeka/Meeting-Summarizer/blob/main/meetsum.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# 🛠️ Install dependencies in Colab
!pip install -q pydub SpeechRecognition requests openai-whisper
!apt-get -qq install ffmpeg

# ✅ Imports
import whisper
import requests
import json
from pydub import AudioSegment
import os

# 🔐 OpenRouter API
OPENROUTER_API_KEY = "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"
MODEL = "mistralai/mistral-7b-instruct:free"

# 🔊 Whisper transcription
def whisper_transcribe(audio_path):
    try:
        model = whisper.load_model("large")  # You can use "small" or "medium" for better results
        result = model.transcribe(audio_path)
        return result["text"]
    except Exception as e:
        return f"Whisper error: {str(e)}"

# 🧠 Summary via OpenRouter
def summarize_meeting(input_text):
    url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json",
        "HTTP-Referer": "https://your-site.com",
        "X-Title": "Meeting Summarizer",
    }

    prompt = f"""
From the following meeting transcript, extract only the task assignments.

Return a JSON array where each object has:
- assigned_to
- department
- task_description
- deadline

Do not include general updates, announcements, welcomes, or summaries.
Only include actual responsibilities or assignments clearly given to individuals or teams.

Meeting Transcript:
{input_text}

"""

    payload = {
        "model": MODEL,
        "messages": [{"role": "user", "content": prompt}]
    }

    response = requests.post(url, headers=headers, data=json.dumps(payload))

    if response.status_code == 200:
        result = response.json()
        return result['choices'][0]['message']['content']
    else:
        return f"API Error: {response.status_code} - {response.text}"

# 🧾 Main function
def main():
    print("📥 How would you like to provide meeting input?")
    print("1. Paste text manually")
    print("2. Upload audio file (.mp3 or .wav)")
    choice = input("Enter 1 or 2: ").strip()

    if choice == "1":
        print("\n✍️ Please paste the meeting transcript below. Press Enter when done.")
        print("(Type END on a new line to finish)")
        lines = []
        while True:
            line = input()
            if line.strip().upper() == "END":
                break
            lines.append(line)
        meeting_text = "\n".join(lines)

    elif choice == "2":
        from google.colab import files
        uploaded = files.upload()
        audio_path = list(uploaded.keys())[0]

        if not (audio_path.endswith(".wav") or audio_path.endswith(".mp3")):
            print("❌ Please upload a .wav or .mp3 file only.")
            return

        # Optional: Convert MP3 to WAV if needed (Whisper supports both)
        if audio_path.endswith(".mp3"):
            sound = AudioSegment.from_mp3(audio_path)
            audio_path = audio_path.replace(".mp3", ".wav")
            sound.export(audio_path, format="wav")

        print("🎧 Transcribing audio using Whisper...")
        meeting_text = whisper_transcribe(audio_path)
        print(f"\n📝 Transcribed Text:\n{meeting_text}")

    else:
        print("❌ Invalid choice. Please enter 1 or 2.")
        return

    print("\n🧠 Sending content for summarization...")
    result = summarize_meeting(meeting_text)
    print("\n📋 Structured Output:\n", result)

# ✅ Start the program
main()


📥 How would you like to provide meeting input?
1. Paste text manually
2. Upload audio file (.mp3 or .wav)
Enter 1 or 2: 2


Saving ttsMP3.com_VoiceText_2025-6-10_12-50-11.mp3 to ttsMP3.com_VoiceText_2025-6-10_12-50-11.mp3
🎧 Transcribing audio using Whisper...

📝 Transcribed Text:
 Good morning everyone, let's get started. First, a quick overview of our quarterly performance, we saw a 12% increase in revenue compared to last quarter, primarily driven by our new enterprise clients in the fintech sector. However, the operations team flagged a rising customer churn rate in our SaaS product line, so Ravi, I'd like you to investigate the retention metrics and propose improvements by next Friday. On that note, Priya from marketing will work closely with you to align messaging and campaign adjustments. Moving on, we've onboarded six new members across departments, please welcome Rajesh in sales, Ananya in data analytics, and the others joining HR and product development. Make sure they're looped into their respective onboarding and buddy sessions. Now, regarding performance, we did a mid-year review, and noticed pr